## The function 'nasa_exo' takes as input the SWEET-Cat table and adds lines to a text file for each of the stars in SWEET-Cat with 4 columns corresponding, in this order, to the value of vsini, the upper bound, the lower bound, the bibcode and the catalog name ('nasa_exoplanet_archive').

In [1]:
# imports

# SWEET-Cat
import urllib
import pandas as pd
import numpy as np
from astropy import coordinates as coord
from astropy import units as u

from astropy.table import Table
from astropy.io.ascii import convert_numpy

# astroquery
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive as nas
from astroquery.utils.tap.core import TapPlus #tap service

In [2]:
#Download SWEET-Cat
sweetCat_table_url = "https://sweetcat.iastro.pt/catalog/SWEETCAT_Dataframe.csv"
converters={'gaia_dr2': [convert_numpy(np.int64)],'gaia_dr3': [convert_numpy(np.int64)] }
sweet_table = Table.read(sweetCat_table_url, encoding='UTF-8',format='csv', converters=converters)

sweet_stars = sweet_table['Name']

# Nasa
nasa_exo = TapPlus(url="https://exoplanetarchive.ipac.caltech.edu/TAP/")

In [ ]:
def nasa_exo(sweet_table):
    
    sweet_stars = sweet_table['Name']
    
    gaia = sweet_table['gaia_dr2']
    
    identifiers = []
    for i in gaia:
        identifiers.append("Gaia DR2 " + str(i))
    
    arch = nas.query_criteria(table="ps")
    
    result_nas = arch[np.in1d(arch['gaia_id'], identifiers)]
    
    result_nasa = result_nasa["hostname","st_vsin","st_vsinerr1","st_vsinerr2","pl_refname"]
    
    #total dataframe
    df_nasa = result_nasa.to_pandas()
    df = df.drop(['hostname'], axis=1)
    cat = ['nasa_exoplanet_archive']*len(result_nasa)
    df['Catalogue'] = cat
    df = df.drop_duplicates()
    
    #dataframe for each star
    for star_name in sweet_stars:
        gaia2_id = sweet_table[sweet_table['Name'] == star_name]['gaia_dr2']
        star_id2 = "Gaia DR2 " + str(gaia2_id[0])
        df_star = df[df['gaia_id'] == star_id2]
    
        # Make csv file with the values
        if "/" in star_name or "*" in star_name:
            file_name = star_id2
        else:
            file_name = star_name

        df_star.to_csv('{}.txt'.format(file_name), mode='a', index=False, header=False, sep = '\t')
    
    return 'nasa done'